In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
# Import training data csv and split into train and test dataframes
X_df = pd.read_csv('./data/train.csv', low_memory=False)
y_df = X_df['SalePrice']
X_df = X_df.drop(['SalePrice'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.3, random_state=0)
combine = [X_train, X_test]

In [3]:
X_train.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
165083,1588882,1235201,14304,132,1.0,2007,NaN,NaN,3/31/2011 0:00,310SJ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
343063,2369725,1784631,11596,136,NaN,1000,0.0,NaN,5/29/2008 0:00,EX200LC5,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
196409,1634391,1334772,4794,132,1.0,1999,NaN,NaN,5/22/2007 0:00,710D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121664,1464164,1212433,7057,132,2.0,1995,NaN,NaN,5/13/2000 0:00,312,...,20 inch,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
187833,1623733,1101315,4763,132,4.0,1992,NaN,NaN,2/11/2009 0:00,650G,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,None or Unspecified,NaN,NaN


In [4]:
y_train.head()

165083    37000
343063    20000
196409    18000
121664    39000
187833    18000
Name: SalePrice, dtype: int64

In [5]:
X_train.shape

(280787, 52)

In [6]:
# Delete all columns who have more than 50% of their data missing
for column in X_train:
    if X_train[column].isnull().sum()/280787 > .5:
        X_train = X_train.drop([column], axis=1)
        X_test = X_test.drop([column], axis=1)

X_train.shape

(280787, 17)

In [7]:
# We have now reduced our feature count from 52 down to 17.
# Lets take a look and see which remaining features have > 30% of data missing

for column in X_train:
    if X_train[column].isnull().sum()/280787 > .3:
        print(column)
        
# Coupler has Null values and strings of None or Unspecified. 
# fiSecondaryDesc in many cases is present inside of the 'fiModelDesc. 
#     - I will leave it in for now but can probably discard it as well

X_train = X_train.drop(["Coupler"], axis=1)
X_test = X_test.drop(["Coupler"], axis=1)

fiSecondaryDesc
Coupler


In [8]:
# Lets see what remaining data is missing > 5%
for column in X_train:
    if X_train[column].isnull().sum()/280787 > .05:
        print(column)

auctioneerID
fiSecondaryDesc
Hydraulics


In [9]:
print(X_train['Hydraulics'].unique())
print(X_train['Hydraulics'].isnull().sum()/280787)

# 20% NA seems high but I will go ahead and keep the Hydraulic column and create dummy variables for it.
# First I want to see how many true NA we have though since there are rows with None or Unspecified 

[nan 'Auxiliary' 'Standard' '2 Valve' 'Base + 1 Function' '3 Valve'
 'Base + 6 Function' '4 Valve' 'Base + 2 Function' 'Base + 4 Function'
 'Base + 3 Function' 'Base + 5 Function' 'None or Unspecified']
0.200628946497


In [10]:
X_train['Hydraulics'] = X_train['Hydraulics'].astype(object).fillna('None or Unspecified')
X_test['Hydraulics'] = X_test['Hydraulics'].astype(object).fillna('None or Unspecified')
    
train_objs_num = len(X_train)
dataset = pd.concat(objs=[X_train, X_test], axis=0)
dataset = pd.get_dummies(data=dataset, columns=["Hydraulics"])
X_train = dataset[:train_objs_num]
X_test = dataset[train_objs_num:]

In [11]:
# I don't want my model to fit for auctioneerID because it will reduce how scalable the model will be in its ability to predict auction prices with unseen auctioneers

X_train = X_train.drop(['auctioneerID'], axis=1)
X_test = X_test.drop(['auctioneerID'], axis=1)

In [12]:
# Lets see what my datatypes are to get an idea of what features I still need to clean
X_train.dtypes

SalesID                            int64
MachineID                          int64
ModelID                            int64
datasource                         int64
YearMade                           int64
saledate                          object
fiModelDesc                       object
fiBaseModel                       object
fiSecondaryDesc                   object
fiProductClassDesc                object
state                             object
ProductGroup                      object
ProductGroupDesc                  object
Enclosure                         object
Hydraulics_2 Valve                 uint8
Hydraulics_3 Valve                 uint8
Hydraulics_4 Valve                 uint8
Hydraulics_Auxiliary               uint8
Hydraulics_Base + 1 Function       uint8
Hydraulics_Base + 2 Function       uint8
Hydraulics_Base + 3 Function       uint8
Hydraulics_Base + 4 Function       uint8
Hydraulics_Base + 5 Function       uint8
Hydraulics_Base + 6 Function       uint8
Hydraulics_None 

In [13]:
# Lets convert the saledate to just the year

from dateutil.parser import parse

saledate_array = X_train['saledate'].as_matrix()
updated_series = pd.Series([ parse(date).year for date in saledate_array ])
X_train['saledate'] = updated_series

saledate_array = X_test['saledate'].as_matrix()
updated_series = pd.Series([ parse(date).year for date in saledate_array ])
X_test['saledate'] = updated_series


In [14]:
X_train.head()

,SalesID,MachineID,ModelID,datasource,YearMade,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiProductClassDesc,...,Hydraulics_4 Valve,Hydraulics_Auxiliary,Hydraulics_Base + 1 Function,Hydraulics_Base + 2 Function,Hydraulics_Base + 3 Function,Hydraulics_Base + 4 Function,Hydraulics_Base + 5 Function,Hydraulics_Base + 6 Function,Hydraulics_None or Unspecified,Hydraulics_Standard
165083,1588882,1235201,14304,132,2007,2011.0,310SJ,310,SJ,Backhoe Loader - 0.0 to 14.0 Ft Standard Diggi...,...,0,0,0,0,0,0,0,0,1,0
343063,2369725,1784631,11596,136,1000,NaN,EX200LC5,EX200,NaN,"Hydraulic Excavator, Track - 19.0 to 21.0 Metr...",...,0,1,0,0,0,0,0,0,0,0
196409,1634391,1334772,4794,132,1999,2011.0,710D,710,D,Backhoe Loader - 16.0 + Ft Standard Digging Depth,...,0,0,0,0,0,0,0,0,1,0
121664,1464164,1212433,7057,132,1995,2007.0,312,312,NaN,"Hydraulic Excavator, Track - 12.0 to 14.0 Metr...",...,0,0,0,0,0,0,0,0,0,1
187833,1623733,1101315,4763,132,1992,2010.0,650G,650,G,"Track Type Tractor, Dozer - 85.0 to 105.0 Hors...",...,0,0,0,0,0,0,0,0,0,0


In [15]:
# It doesn't make sense to fill the na's with 0 so instead I will fill these with the mean 
X_train['saledate'][X_train['saledate'] == 0] = np.NaN
X_test['saledate'][X_test['saledate'] == 0] = np.NaN
print(X_train['saledate'].mean(skipna=True))

2004.0955102830972


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
X_train['saledate'] = X_train['saledate'].fillna(2004)
X_test['saledate'] = X_test['saledate'].fillna(2004)

X_train['saledate'] = X_train['saledate'].astype(int)
X_test['saledate'] = X_test['saledate'].astype(int)
X_train['saledate'].unique()

array([2011, 2004, 2007, 2010, 2008, 1996, 2009, 2005, 1991, 1993, 1998,
       2000, 2003, 2001, 1997, 1995, 2006, 2002, 1992, 1990, 1989, 1994,
       1999])

In [17]:
# fiModelDesc and fiBaseModel are very similar. Which has more data?
print(X_train['fiModelDesc'].isnull().sum()/280787)
print(X_train['fiBaseModel'].isnull().sum()/280787)

# They both have 100% data!

0.0
0.0


In [18]:
# Which has more unique values?
print(X_train['fiModelDesc'].unique().size)
print(X_train['fiBaseModel'].unique().size)

4685
1856


In [19]:
# Lets drop fiModelDesc and later turn fiBaseModel into a category data type
X_train = X_train.drop(['fiModelDesc'], axis=1)
X_test = X_test.drop(['fiModelDesc'], axis=1)

In [20]:
# How bout these two other features? How many null values are there?
print(X_train['fiSecondaryDesc'].isnull().sum()/280787)
print(X_train['fiProductClassDesc'].isnull().sum()/280787)

0.341892608988
0.0


In [21]:
# 34% of fiSecondaryDesc is missing. Will get rid of this too.
X_train = X_train.drop(['fiSecondaryDesc'], axis=1)
X_test = X_test.drop(['fiSecondaryDesc'], axis=1)

In [22]:
X_train.head()

,SalesID,MachineID,ModelID,datasource,YearMade,saledate,fiBaseModel,fiProductClassDesc,state,ProductGroup,...,Hydraulics_4 Valve,Hydraulics_Auxiliary,Hydraulics_Base + 1 Function,Hydraulics_Base + 2 Function,Hydraulics_Base + 3 Function,Hydraulics_Base + 4 Function,Hydraulics_Base + 5 Function,Hydraulics_Base + 6 Function,Hydraulics_None or Unspecified,Hydraulics_Standard
165083,1588882,1235201,14304,132,2007,2011,310,Backhoe Loader - 0.0 to 14.0 Ft Standard Diggi...,Georgia,BL,...,0,0,0,0,0,0,0,0,1,0
343063,2369725,1784631,11596,136,1000,2004,EX200,"Hydraulic Excavator, Track - 19.0 to 21.0 Metr...",Ohio,TEX,...,0,1,0,0,0,0,0,0,0,0
196409,1634391,1334772,4794,132,1999,2011,710,Backhoe Loader - 16.0 + Ft Standard Digging Depth,California,BL,...,0,0,0,0,0,0,0,0,1,0
121664,1464164,1212433,7057,132,1995,2007,312,"Hydraulic Excavator, Track - 12.0 to 14.0 Metr...",New York,TEX,...,0,0,0,0,0,0,0,0,0,1
187833,1623733,1101315,4763,132,1992,2010,650,"Track Type Tractor, Dozer - 85.0 to 105.0 Hors...",Florida,TTT,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# Lets map all the states to a region

# Region 1: Northeast
# Division 1: New England (Connecticut, Maine, Massachusetts, New Hampshire, Rhode Island, and Vermont)
# Division 2: Mid-Atlantic (New Jersey, New York, and Pennsylvania)

# Region 2: Midwest
# Division 3: East North Central (Illinois, Indiana, Michigan, Ohio, and Wisconsin)
# Division 4: West North Central (Iowa, Kansas, Minnesota, Missouri, Nebraska, North Dakota, and South Dakota)
# Region 3: South
# Division 5: South Atlantic (Delaware, Florida, Georgia, Maryland, North Carolina, South Carolina, Virginia, District of Columbia, and West Virginia)
# Division 6: East South Central (Alabama, Kentucky, Mississippi, and Tennessee)
# Division 7: West South Central (Arkansas, Louisiana, Oklahoma, and Texas)
# Region 4: West
# Division 8: Mountain (Arizona, Colorado, Idaho, Montana, Nevada, New Mexico, Utah, and Wyoming)
# Division 9: Pacific (Alaska, California, Hawaii, Oregon, and Washington)

state_mapping = {'Alabama':3, 'North Carolina':3, 'New York':1, 'Texas':3, 'Arizona':4,
       'Florida':3, 'Illinois':2, 'Oregon':4, 'Ohio':2, 'Arkansas':3, 'Wisconsin':2,
       'Kansas':2, 'Nevada':4, 'Iowa':2, 'Maine':1, 'Massachusetts':1, 'California':4,
       'Louisiana':3, 'Minnesota':2, 'New Hampshire':1, 'Idaho':4, 'Michigan':2,
       'Mississippi':3, 'Georgia':3, 'Missouri':2, 'South Carolina':3, 'Utah':4,
       'Tennessee':3, 'Washington':4, 'Virginia':3, 'South Dakota':2,
       'West Virginia':3, 'Oklahoma':3, 'Pennsylvania':1, 'Wyoming':4,
       'North Dakota':2, 'New Jersey':1, 'Kentucky':3, 'Montana':4, 'Alaska':4,
       'Nebraska':2, 'Maryland':3, 'Hawaii':4, 'Colorado':4, 'New Mexico':4,
       'Indiana':2, 'Connecticut':1, 'Delaware':3, 'Rhode Island':1, 'Vermont':1,
       'Washington DC':3, 'Puerto Rico':3, 'Unspecified':0}

X_train['state'] = X_train['state'].map(state_mapping)
X_train['state'] = X_train['state'].fillna(0)
X_train['state'] = X_train['state'].astype(int)

X_test['state'] = X_test['state'].map(state_mapping)
X_test['state'] = X_test['state'].fillna(0)
X_test['state'] = X_test['state'].astype(int)
    
X_train.head()

,SalesID,MachineID,ModelID,datasource,YearMade,saledate,fiBaseModel,fiProductClassDesc,state,ProductGroup,...,Hydraulics_4 Valve,Hydraulics_Auxiliary,Hydraulics_Base + 1 Function,Hydraulics_Base + 2 Function,Hydraulics_Base + 3 Function,Hydraulics_Base + 4 Function,Hydraulics_Base + 5 Function,Hydraulics_Base + 6 Function,Hydraulics_None or Unspecified,Hydraulics_Standard
165083,1588882,1235201,14304,132,2007,2011,310,Backhoe Loader - 0.0 to 14.0 Ft Standard Diggi...,3,BL,...,0,0,0,0,0,0,0,0,1,0
343063,2369725,1784631,11596,136,1000,2004,EX200,"Hydraulic Excavator, Track - 19.0 to 21.0 Metr...",2,TEX,...,0,1,0,0,0,0,0,0,0,0
196409,1634391,1334772,4794,132,1999,2011,710,Backhoe Loader - 16.0 + Ft Standard Digging Depth,4,BL,...,0,0,0,0,0,0,0,0,1,0
121664,1464164,1212433,7057,132,1995,2007,312,"Hydraulic Excavator, Track - 12.0 to 14.0 Metr...",1,TEX,...,0,0,0,0,0,0,0,0,0,1
187833,1623733,1101315,4763,132,1992,2010,650,"Track Type Tractor, Dozer - 85.0 to 105.0 Hors...",3,TTT,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# Lets see if we can create more dummy variables

print(X_train['ProductGroup'].unique())
print(X_train['ProductGroupDesc'].unique())

['BL' 'TEX' 'TTT' 'SSL' 'WL' 'MG']
['Backhoe Loaders' 'Track Excavators' 'Track Type Tractors'
 'Skid Steer Loaders' 'Wheel Loader' 'Motor Graders']


In [25]:
train_objs_num = len(X_train)
dataset = pd.concat(objs=[X_train, X_test], axis=0)
dataset = pd.get_dummies(data=dataset, columns=["ProductGroup"])
dataset = pd.get_dummies(data=dataset, columns=["ProductGroupDesc"])
X_train = dataset[:train_objs_num]
X_test = dataset[train_objs_num:]

In [26]:
# It looks like Enclosure could also be converted to a dummy
print(X_train['Enclosure'].unique())

['OROPS' 'EROPS' 'EROPS w AC' nan 'NO ROPS' 'EROPS AC'
 'None or Unspecified']


In [27]:
# Fill in all the na rows to match the None or Unspecified string
X_train['Enclosure'] = X_train['Enclosure'].astype(object).fillna('None or Unspecified')
X_test['Enclosure'] = X_test['Enclosure'].astype(object).fillna('None or Unspecified')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
print(X_train['Enclosure'].unique())

['OROPS' 'EROPS' 'EROPS w AC' 'None or Unspecified' 'NO ROPS' 'EROPS AC']


In [29]:
# Convert the Enclosure values to dummy variables
train_objs_num = len(X_train)
dataset = pd.concat(objs=[X_train, X_test], axis=0)
dataset = pd.get_dummies(data=dataset, columns=["Enclosure"])
X_train = dataset[:train_objs_num]
X_test = dataset[train_objs_num:]

In [30]:
# Might be too many to convert to dummy but for my first pass I will try it 
X_train['fiProductClassDesc'].unique()

array(['Backhoe Loader - 0.0 to 14.0 Ft Standard Digging Depth',
       'Hydraulic Excavator, Track - 19.0 to 21.0 Metric Tons',
       'Backhoe Loader - 16.0 + Ft Standard Digging Depth',
       'Hydraulic Excavator, Track - 12.0 to 14.0 Metric Tons',
       'Track Type Tractor, Dozer - 85.0 to 105.0 Horsepower',
       'Skid Steer Loader - 1351.0 to 1601.0 Lb Operating Capacity',
       'Backhoe Loader - 14.0 to 15.0 Ft Standard Digging Depth',
       'Backhoe Loader - 15.0 to 16.0 Ft Standard Digging Depth',
       'Track Type Tractor, Dozer - 75.0 to 85.0 Horsepower',
       'Skid Steer Loader - 1251.0 to 1351.0 Lb Operating Capacity',
       'Wheel Loader - 120.0 to 135.0 Horsepower',
       'Wheel Loader - 150.0 to 175.0 Horsepower',
       'Wheel Loader - 275.0 to 350.0 Horsepower',
       'Motorgrader - 45.0 to 130.0 Horsepower',
       'Hydraulic Excavator, Track - 28.0 to 33.0 Metric Tons',
       'Hydraulic Excavator, Track - 2.0 to 3.0 Metric Tons',
       'Hydraulic Excava

In [31]:
# Covert fiProductClassDesc to dummy variables

train_objs_num = len(X_train)
dataset = pd.concat(objs=[X_train, X_test], axis=0)
dataset = pd.get_dummies(data=dataset, columns=["fiProductClassDesc"])
X_train = dataset[:train_objs_num]
X_test = dataset[train_objs_num:]

In [32]:
# Drop fiBaseModel because there are too many different data values to use it for any sort of categorization or dummy data
# I tried to convert it to a dummy variable but it made the model highly complex and produced a negative score
# Lets try again with pre-processing
# Preprocessing this variable had a minor improvement. Lets keep it in.

# X_train = X_train.drop("fiBaseModel", axis=1)
# X_test = X_test.drop("fiBaseModel", axis=1)

In [33]:
X_train.head()

,SalesID,MachineID,ModelID,datasource,YearMade,saledate,fiBaseModel,state,Hydraulics_2 Valve,Hydraulics_3 Valve,...,fiProductClassDesc_Wheel Loader - 225.0 to 250.0 Horsepower,fiProductClassDesc_Wheel Loader - 250.0 to 275.0 Horsepower,fiProductClassDesc_Wheel Loader - 275.0 to 350.0 Horsepower,fiProductClassDesc_Wheel Loader - 350.0 to 500.0 Horsepower,fiProductClassDesc_Wheel Loader - 40.0 to 60.0 Horsepower,fiProductClassDesc_Wheel Loader - 500.0 to 1000.0 Horsepower,fiProductClassDesc_Wheel Loader - 60.0 to 80.0 Horsepower,fiProductClassDesc_Wheel Loader - 80.0 to 90.0 Horsepower,fiProductClassDesc_Wheel Loader - 90.0 to 100.0 Horsepower,fiProductClassDesc_Wheel Loader - Unidentified
165083,1588882,1235201,14304,132,2007,2011,310,3,0,0,...,0,0,0,0,0,0,0,0,0,0
343063,2369725,1784631,11596,136,1000,2004,EX200,2,0,0,...,0,0,0,0,0,0,0,0,0,0
196409,1634391,1334772,4794,132,1999,2011,710,4,0,0,...,0,0,0,0,0,0,0,0,0,0
121664,1464164,1212433,7057,132,1995,2007,312,1,0,0,...,0,0,0,0,0,0,0,0,0,0
187833,1623733,1101315,4763,132,1992,2010,650,3,1,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
# There is corrupted data where the year 1000 is showing up repeatedly
X_train['YearMade'].unique()

array([2007, 1000, 1999, 1995, 1992, 2000, 2006, 2005, 1984, 1987, 1996,
       2003, 1988, 1993, 1998, 1973, 1962, 1969, 1966, 2001, 1997, 1989,
       1986, 1970, 2002, 1994, 1990, 1979, 1981, 1985, 1975, 1972, 1976,
       2004, 1983, 1978, 1980, 1991, 1956, 2008, 1977, 1974, 1982, 1967,
       1971, 1965, 1964, 1968, 1963, 2011, 2009, 1919, 1961, 1920, 1960,
       1959, 1954, 2010, 1950, 1951, 1958, 1957, 1955, 1952, 1948, 1953,
       1937, 1942])

In [35]:
# Clean this value and make it NaN so we can get the mean and replace the values with the mean
X_train['YearMade'][X_train['YearMade'] == 1000] = np.NaN
X_test['YearMade'][X_test['YearMade'] == 1000] = np.NaN
print(X_train['YearMade'].mean(skipna=True))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5233: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2862: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py

1993.7359019154194


In [36]:
X_train['YearMade'] = X_train['YearMade'].fillna(1994)
X_test['YearMade'] = X_test['YearMade'].fillna(1994)

X_train['YearMade'] = X_train['YearMade'].astype(int)
X_test['YearMade'] = X_test['YearMade'].astype(int)
X_train['YearMade'].unique()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

array([2007, 1994, 1999, 1995, 1992, 2000, 2006, 2005, 1984, 1987, 1996,
       2003, 1988, 1993, 1998, 1973, 1962, 1969, 1966, 2001, 1997, 1989,
       1986, 1970, 2002, 1990, 1979, 1981, 1985, 1975, 1972, 1976, 2004,
       1983, 1978, 1980, 1991, 1956, 2008, 1977, 1974, 1982, 1967, 1971,
       1965, 1964, 1968, 1963, 2011, 2009, 1919, 1961, 1920, 1960, 1959,
       1954, 2010, 1950, 1951, 1958, 1957, 1955, 1952, 1948, 1953, 1937,
       1942])

In [37]:
X_train.head()

,SalesID,MachineID,ModelID,datasource,YearMade,saledate,fiBaseModel,state,Hydraulics_2 Valve,Hydraulics_3 Valve,...,fiProductClassDesc_Wheel Loader - 225.0 to 250.0 Horsepower,fiProductClassDesc_Wheel Loader - 250.0 to 275.0 Horsepower,fiProductClassDesc_Wheel Loader - 275.0 to 350.0 Horsepower,fiProductClassDesc_Wheel Loader - 350.0 to 500.0 Horsepower,fiProductClassDesc_Wheel Loader - 40.0 to 60.0 Horsepower,fiProductClassDesc_Wheel Loader - 500.0 to 1000.0 Horsepower,fiProductClassDesc_Wheel Loader - 60.0 to 80.0 Horsepower,fiProductClassDesc_Wheel Loader - 80.0 to 90.0 Horsepower,fiProductClassDesc_Wheel Loader - 90.0 to 100.0 Horsepower,fiProductClassDesc_Wheel Loader - Unidentified
165083,1588882,1235201,14304,132,2007,2011,310,3,0,0,...,0,0,0,0,0,0,0,0,0,0
343063,2369725,1784631,11596,136,1994,2004,EX200,2,0,0,...,0,0,0,0,0,0,0,0,0,0
196409,1634391,1334772,4794,132,1999,2011,710,4,0,0,...,0,0,0,0,0,0,0,0,0,0
121664,1464164,1212433,7057,132,1995,2007,312,1,0,0,...,0,0,0,0,0,0,0,0,0,0
187833,1623733,1101315,4763,132,1992,2010,650,3,1,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
# Feature engineer 'equipment age'
X_train['equipment_age'] = X_train['saledate'] - X_train['YearMade']
X_test['equipment_age'] = X_test['saledate'] - X_test['YearMade']


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [39]:
X_train.head()

,SalesID,MachineID,ModelID,datasource,YearMade,saledate,fiBaseModel,state,Hydraulics_2 Valve,Hydraulics_3 Valve,...,fiProductClassDesc_Wheel Loader - 250.0 to 275.0 Horsepower,fiProductClassDesc_Wheel Loader - 275.0 to 350.0 Horsepower,fiProductClassDesc_Wheel Loader - 350.0 to 500.0 Horsepower,fiProductClassDesc_Wheel Loader - 40.0 to 60.0 Horsepower,fiProductClassDesc_Wheel Loader - 500.0 to 1000.0 Horsepower,fiProductClassDesc_Wheel Loader - 60.0 to 80.0 Horsepower,fiProductClassDesc_Wheel Loader - 80.0 to 90.0 Horsepower,fiProductClassDesc_Wheel Loader - 90.0 to 100.0 Horsepower,fiProductClassDesc_Wheel Loader - Unidentified,equipment_age
165083,1588882,1235201,14304,132,2007,2011,310,3,0,0,...,0,0,0,0,0,0,0,0,0,4
343063,2369725,1784631,11596,136,1994,2004,EX200,2,0,0,...,0,0,0,0,0,0,0,0,0,10
196409,1634391,1334772,4794,132,1999,2011,710,4,0,0,...,0,0,0,0,0,0,0,0,0,12
121664,1464164,1212433,7057,132,1995,2007,312,1,0,0,...,0,0,0,0,0,0,0,0,0,12
187833,1623733,1101315,4763,132,1992,2010,650,3,1,0,...,0,0,0,0,0,0,0,0,0,18


In [40]:
X_train = X_train.drop(['SalesID', 'MachineID'], axis=1)
X_test = X_test.drop(['SalesID', 'MachineID'], axis=1)

In [41]:
X_train.dtypes

ModelID                                                                      int64
datasource                                                                   int64
YearMade                                                                     int64
saledate                                                                     int64
fiBaseModel                                                                 object
state                                                                        int64
Hydraulics_2 Valve                                                           uint8
Hydraulics_3 Valve                                                           uint8
Hydraulics_4 Valve                                                           uint8
Hydraulics_Auxiliary                                                         uint8
Hydraulics_Base + 1 Function                                                 uint8
Hydraulics_Base + 2 Function                                                 uint8
Hydr

In [42]:
print(X_train.shape)
print(X_test.shape)

(280787, 111)
(120338, 111)


In [43]:
X_train.head()

,ModelID,datasource,YearMade,saledate,fiBaseModel,state,Hydraulics_2 Valve,Hydraulics_3 Valve,Hydraulics_4 Valve,Hydraulics_Auxiliary,...,fiProductClassDesc_Wheel Loader - 250.0 to 275.0 Horsepower,fiProductClassDesc_Wheel Loader - 275.0 to 350.0 Horsepower,fiProductClassDesc_Wheel Loader - 350.0 to 500.0 Horsepower,fiProductClassDesc_Wheel Loader - 40.0 to 60.0 Horsepower,fiProductClassDesc_Wheel Loader - 500.0 to 1000.0 Horsepower,fiProductClassDesc_Wheel Loader - 60.0 to 80.0 Horsepower,fiProductClassDesc_Wheel Loader - 80.0 to 90.0 Horsepower,fiProductClassDesc_Wheel Loader - 90.0 to 100.0 Horsepower,fiProductClassDesc_Wheel Loader - Unidentified,equipment_age
165083,14304,132,2007,2011,310,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
343063,11596,136,1994,2004,EX200,2,0,0,0,1,...,0,0,0,0,0,0,0,0,0,10
196409,4794,132,1999,2011,710,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
121664,7057,132,1995,2007,312,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
187833,4763,132,1992,2010,650,3,1,0,0,0,...,0,0,0,0,0,0,0,0,0,18


In [44]:
X_train['ModelID'] = X_train['ModelID'].astype('category')
X_train['datasource'] = X_train['datasource'].astype('category')
X_train['state'] = X_train['state'].astype('category')
X_test['ModelID'] = X_test['ModelID'].astype('category')
X_test['datasource'] = X_test['datasource'].astype('category')
X_test['state'] = X_test['state'].astype('category')

In [47]:
# Last score before deleting fiProductClassDesc dummies was: 0.58793037029487993
# I deleted to see how score would be effected because 
# fiProductClassDesc would be very difficult to format and clean for future test data and makes the model highly dimensional

# After dropping the score dropped from 0.5879 to 0.413
# I decided to re-introduce these columns

In [48]:
# coef_df = pd.DataFrame({'Coef_': linear.coef_, 'Feature': X_train.columns})
# coef_df.sort_values(by="Coef_", ascending=True)

In [49]:
# from sklearn.model_selection import cross_val_score
# linear = LinearRegression()
# cross_val_score(linear, X_train, y_train, n_jobs=-1)

In [50]:
from sklearn.linear_model import Lasso
# Simplifies the model by only using the most important features
# Grid search for alpha, lower alpha uses more fetures
# Ex: .0001, .001, .01, 1
# With alpha = 0.1 I got the same score
# lasso = Lasso(alpha=.01)
# lasso.fit(X_train, y_train)
# lasso.score(X_test, y_test)

In [51]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
X_train['fiBaseModel'] = le.fit_transform(X_train['fiBaseModel'])
X_test['fiBaseModel'] = le.fit_transform(X_test['fiBaseModel'])

In [52]:
from sklearn.linear_model import LinearRegression
# linear = LinearRegression()
# linear.fit(X_train, y_train)
# linear.score(X_test, y_test)

In [53]:
# There was a slight improvement (58.79 to 59.3%) in using Label Encoder. Where else could I use this?
# Lets try encoding fiProductClassDesc instead of dummy variables 
# le = preprocessing.LabelEncoder()
# X_train['fiProductClassDesc'] = le.fit_transform(X_train['fiProductClassDesc'])
# X_test['fiProductClassDesc'] = le.fit_transform(X_test['fiProductClassDesc'])

# This hurt the score significantly? Why?
# Knocked it down from 59.3% to 41.7%

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

param_grid = {"alpha": [.0001, .001, .01, .1, 1]}

grid_search = GridSearchCV(Lasso(max_iter=-1), param_grid)
grid_search.fit(X_train, y_train)
grid_search.score(X_test, y_test)